In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have defined df, max_len, and tok (Tokenizer) earlier

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=101)

X_train_mat = tok.texts_to_sequences(X_train)
X_test_mat = tok.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_mat, maxlen=max_len)
X_test_padded = pad_sequences(X_test_mat, maxlen=max_len)

# Assuming you have trained and obtained a model named model_bi_lstm
loaded_model = load_model('gru_model.h5')

# Predict using the trained model on test data
y_pred_prob = loaded_model.predict(X_test_padded, batch_size=128)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(y_test, y_pred) if true_label == pred_label)
accuracy = correct_predictions / len(y_pred) * 100

print('Correct Predictions:', correct_predictions)
print('Wrong Predictions:', len(y_pred) - correct_predictions)
print('Accuracy: {:.2f}%'.format(accuracy))


NameError: name 'df' is not defined

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense, Dropout, Conv1D, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import time

In [7]:
tf.config.run_functions_eagerly(True)

In [8]:
df = pd.read_csv('IMDB.csv')

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
df = df[['review', 'sentiment']]

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
# Data preprocessing
blanks = []
for i, rv, sn in df.itertuples():
    if type(rv) == str and rv.isspace():
        blanks.append(i)
        
if len(blanks) > 0:
    print(f'There are {len(blanks)} empty space strings in the dataset')
    df.drop(blanks, inplace=True)

In [11]:
df.nunique()

review       49582
sentiment        2
dtype: int64

In [12]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [13]:
max_seq_length = df['review'].str.len().max()

print("Max sequence length:", max_seq_length)

Max sequence length: 13704


In [14]:
words = set(word.lower() for sentence in df['review'] for word in sentence.split())
n_unique_words = len(words)

print("Unique words:", n_unique_words)

Unique words: 390931


In [15]:
X = df['review']
y = df['sentiment']

display(X, y)

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

display(y)

array([1, 1, 1, ..., 0, 0, 0])

In [17]:
# Tokenization
max_len = 150
num_words = 1000

tok = Tokenizer(num_words=num_words)
tok.fit_on_texts(df['review'])

In [18]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=101)

X_train_mat = tok.texts_to_sequences(X_train)
X_test_mat = tok.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_mat, maxlen=max_len)
X_test_padded = pad_sequences(X_test_mat, maxlen=max_len)

In [19]:
# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [20]:
from tensorflow.keras.models import load_model

# Assuming you have trained and obtained a model named model_bi_lstm
loaded_model = load_model('gru_model.h5')
# Predict using the trained model on test data
y_pred_prob = loaded_model.predict(X_test_padded, batch_size=128)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(y_test, y_pred) if true_label == pred_label)
accuracy = correct_predictions / len(y_pred) * 100

print('Correct Predictions:', correct_predictions)
print('Wrong Predictions:', len(y_pred) - correct_predictions)
print('Accuracy: {:.2f}%'.format(accuracy))


C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


118/118 [==============================] - 115s 972ms/step
Correct Predictions: 12990
Wrong Predictions: 2010
Accuracy: 86.60%


In [22]:
from tensorflow.keras.models import load_model

# Assuming you have trained and obtained a model named model_bi_lstm
loaded_model = load_model('imdbmodel/cnn_model.h5')
# Predict using the trained model on test data
y_pred_prob = loaded_model.predict(X_test_padded, batch_size=128)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(y_test, y_pred) if true_label == pred_label)
accuracy = correct_predictions / len(y_pred) * 100

print('Correct Predictions:', correct_predictions)
print('Wrong Predictions:', len(y_pred) - correct_predictions)
print('Accuracy: {:.2f}%'.format(accuracy))


118/118 [==============================] - 3s 22ms/step
Correct Predictions: 12893
Wrong Predictions: 2107
Accuracy: 85.95%


In [23]:
from tensorflow.keras.models import load_model

# Assuming you have trained and obtained a model named model_bi_lstm
loaded_model = load_model('imdbmodel/rnn_model.h5')
# Predict using the trained model on test data
y_pred_prob = loaded_model.predict(X_test_padded, batch_size=128)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(y_test, y_pred) if true_label == pred_label)
accuracy = correct_predictions / len(y_pred) * 100

print('Correct Predictions:', correct_predictions)
print('Wrong Predictions:', len(y_pred) - correct_predictions)
print('Accuracy: {:.2f}%'.format(accuracy))


118/118 [==============================] - 28s 238ms/step
Correct Predictions: 12568
Wrong Predictions: 2432
Accuracy: 83.79%


In [24]:
from tensorflow.keras.models import load_model

# Assuming you have trained and obtained a model named model_bi_lstm
loaded_model = load_model('imdbmodel/lstm_model.h5')
# Predict using the trained model on test data
y_pred_prob = loaded_model.predict(X_test_padded, batch_size=128)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(y_test, y_pred) if true_label == pred_label)
accuracy = correct_predictions / len(y_pred) * 100

print('Correct Predictions:', correct_predictions)
print('Wrong Predictions:', len(y_pred) - correct_predictions)
print('Accuracy: {:.2f}%'.format(accuracy))


118/118 [==============================] - 48s 403ms/step
Correct Predictions: 12840
Wrong Predictions: 2160
Accuracy: 85.60%


In [25]:
from tensorflow.keras.models import load_model

# Assuming you have trained and obtained a model named model_bi_lstm
loaded_model = load_model('imdbmodel/bidirectional_lstm_model.h5')
# Predict using the trained model on test data
y_pred_prob = loaded_model.predict(X_test_padded, batch_size=128)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(y_test, y_pred) if true_label == pred_label)
accuracy = correct_predictions / len(y_pred) * 100

print('Correct Predictions:', correct_predictions)
print('Wrong Predictions:', len(y_pred) - correct_predictions)
print('Accuracy: {:.2f}%'.format(accuracy))


118/118 [==============================] - 88s 748ms/step
Correct Predictions: 12845
Wrong Predictions: 2155
Accuracy: 85.63%


In [26]:
from tensorflow.keras.models import load_model

# Assuming you have trained and obtained a model named model_bi_lstm
loaded_model = load_model('imdbmodel/bidirectional_gru_model.h5')
# Predict using the trained model on test data
y_pred_prob = loaded_model.predict(X_test_padded, batch_size=128)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

# Calculate accuracy
correct_predictions = sum(1 for true_label, pred_label in zip(y_test, y_pred) if true_label == pred_label)
accuracy = correct_predictions / len(y_pred) * 100

print('Correct Predictions:', correct_predictions)
print('Wrong Predictions:', len(y_pred) - correct_predictions)
print('Accuracy: {:.2f}%'.format(accuracy))


118/118 [==============================] - 96s 811ms/step
Correct Predictions: 12970
Wrong Predictions: 2030
Accuracy: 86.47%
